In [13]:
import pandas as pd
import numpy as np

In [61]:
baseball = pd.read_csv("/Users/beckettdowhan/Downloads/baseball_500k.csv", low_memory=False)

In [19]:
# These values will help us determine what happened pitch by pitch
# We can use these values to determine who is on first second and third
PitchCall_unique_values = baseball['PitchCall'].unique()
KorBB_unique_values = baseball['KorBB'].unique()

In [21]:
PitchCall_unique_values

array(['StrikeCalled', 'InPlay', 'BallCalled', 'FoulBall',
       'StrikeSwinging', 'Ball', 'HitByPitch', 'BallIntentional',
       'Undefined', 'BallinDirt'], dtype=object)

In [23]:
KorBB_unique_values

array(['Undefined', 'Walk', 'Strikeout'], dtype=object)

In [33]:
# Create Runner Columns
data = baseball
data['runners_on_1st'] = 0
data['runners_on_2nd'] = 0
data['runners_on_3rd'] = 0

In [35]:
# Runners are assigned bases BEFORE the pitch
# This will make it easier to calculate run matrix in the future
# Go through each combination (8) of runners on for each play outcome
# Very tedious code, but only way to assign runners on
for i in range(0, len(baseball)):
    
    # New inning means no runners on
    if data['PitchofPA'].iloc[i] == 1 and data['PAofInning'].iloc[i] == 1:
        data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]
        continue

    previous_runners = (data['runners_on_1st'].iloc[i-1], data['runners_on_2nd'].iloc[i-1], data['runners_on_3rd'].iloc[i-1])
    
    previous_first = previous_runners[0]
    previous_second = previous_runners[1]
    previous_third = previous_runners[2]

    previous_outs = data['OutsOnPlay'].iloc[i-1]
    previous_runs = data['RunsScored'].iloc[i-1]
    previous_pitch = data['PitchCall'].iloc[i-1]
    previous_KorBB = data['KorBB'].iloc[i-1]



    # What happens for single?
    # Single GUARANTEES a runner on first at the end of the play
    # All outs will go to the lead runner, unless they scored

    # Single has OutsOnPlay = 0 or 1 or 2
    # Single has RunsOnPlay = 0 or 1 or 2
    if data['PlayResult'].iloc[i-1] == 'Single':
        
        # First Combination, done
        if (previous_first == 0) and (previous_second == 0) and (previous_third == 0):
            data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]

        # Second Combination, done
        elif (previous_first == 0) and (previous_second == 1) and (previous_third == 0):
            
            if previous_runs == 1:
                
                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]
                
            elif previous_outs == 1:
                
                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]

            else: 
                
                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 0]          

        # Third Combination, done
        elif (previous_first == 0) and (previous_second == 1) and (previous_third == 1):
            
            if previous_runs == 1:

                if previous_outs == 1:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]
                    
                else:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 1]

            elif previous_runs == 2:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]

            else:

                if previous_outs == 0:
                    
                        data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 1]

                elif previous_outs == 1:

                        data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 0]
                    
        # Fourth Combination, done    
        elif (previous_first == 0) and (previous_second == 0) and (previous_third == 1):
            
            if previous_runs == 1:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]

            elif previous_outs == 1:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]
                
            else: 
    
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 1] 
                
        #Fifth Combination, done
        elif (previous_first == 1) and (previous_second == 0) and (previous_third == 0):
            
            if previous_runs == 1:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]

            elif previous_outs == 1:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]

            else: 
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 0]
                
        # Sixth Combination, done
        elif (previous_first == 1) and (previous_second == 1) and (previous_third == 0):
            
            if previous_runs == 1:

                if previous_outs == 1:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]
                    
                else:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 1]

            elif previous_runs == 2:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]

            else:

                if previous_outs == 1:
                    
                        data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 0]
    
                elif previous_outs == 2:
                    
                        data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]
    
                else:
                    
                        data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 1]

        # Seventh Combination, done
        elif (previous_first == 1) and (previous_second == 0) and (previous_third == 1):

            if previous_runs == 1:

                if previous_outs == 1:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]
                    
                else:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 0]

            elif previous_runs == 2:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]   

            else:

                if previous_outs == 1:
                    
                        data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 1]
    
                elif previous_outs == 2:
                    
                        data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]
    
                elif previous_outs == 0:
                    
                        data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 1]

        # Eighth Combination, done
        elif (previous_first == 1) and (previous_second == 1) and (previous_third == 1):

            if previous_runs == 1:

                if previous_outs == 1:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 0]

                elif previous_outs == 2:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]
                    
                else:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 1]

            elif previous_runs == 2:

                if previous_outs == 1:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]

                else:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 0]

            elif previous_runs == 3:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]

            else:

                if previous_outs == 1:
                    
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 1]

                elif previous_outs == 2:
                    
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 0]


    # Double has OutsOnPlay = 0 or 1
    # Double has RunsOnPlay = 0 or 1 or 2 or 3
    elif data['PlayResult'].iloc[i-1] == 'Double':
         
        # First Combination, done
        if (previous_first == 0) and (previous_second == 0) and (previous_third == 0):
            
            data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 0]

        # Second Combination, done
        elif (previous_first == 0) and (previous_second == 1) and (previous_third == 0):
            
            if previous_runs == 1:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 0]

            elif previous_outs == 1:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 0]

            else: 
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 1]         
                
        # Third Combination, done
        elif (previous_first == 0) and (previous_second == 1) and (previous_third == 1):
            
            if previous_runs == 1:

                if previous_outs == 1:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 0]
                    
                else:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 1]

            elif previous_runs == 2:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 0]

            elif previous_outs == 1 and previous_runs == 0:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 1]

            elif previous_outs == 2:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 0]
                
        # Fourth Combination, done
        elif (previous_first == 0) and (previous_second == 0) and (previous_third == 1):
            if previous_runs == 1:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 0]

            elif previous_outs == 1:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 0]
                
            else: 
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 1]
                
        #Fifth Combination, done
        elif (previous_first == 1) and (previous_second == 0) and (previous_third == 0):
            
            if previous_runs == 1:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 0]

            elif previous_outs == 1:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 0]

            else: 
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 1]
                
        # Sixth Combination, done
        elif (previous_first == 1) and (previous_second == 1) and (previous_third == 0):
            
            if previous_runs == 1:

                if previous_outs == 1:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 0]
                    
                else:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 1]

            elif previous_runs == 2:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 0]

            elif previous_outs == 1 and previous_runs == 0:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 1]

            elif previous_outs == 2 and previous_runs == 0:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 0]

        # Seventh Combination, done
        elif (previous_first == 1) and (previous_second == 0) and (previous_third == 1):

            if previous_runs == 1:

                if previous_outs == 1:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 0]
                    
                else:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 1]

            elif previous_runs == 2:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 0]    

            elif previous_outs == 1 and previous_runs == 0:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 1]

            elif previous_outs == 2 and previous_runs == 0:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 0]

        # Eighth Combination, done
        elif (previous_first == 1) and (previous_second == 1) and (previous_third == 1):

            if previous_runs == 1:

                if previous_outs == 1:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 1]

                elif previous_outs == 2:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 0]

            elif previous_runs == 2:

                if previous_outs == 1:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 1]

                else:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 1]

            elif previous_runs == 3:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 0]

            #Runs scored = 0
            else:
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 1]

    elif data['PlayResult'].iloc[i-1] == 'Triple':
        
        data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 1] 

    elif data['PlayResult'].iloc[i-1] == 'HomeRun':

        data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0] 

    # All Errors have previous_outs = 0
    # Some Errors have previous_runs > 0
    elif data['PlayResult'].iloc[i-1] == 'Error':

        # First Combination, done
        if (previous_first == 0) and (previous_second == 0) and (previous_third == 0):

            if previous_runs == 1:
            
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]

            else: 
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]

        # Second Combination, done
        elif (previous_first == 0) and (previous_second == 1) and (previous_third == 0):
            
            if previous_runs == 1:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 0]

            elif previous_runs == 2:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]   
                
            else:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 1]  
                
        # Third Combination, done
        elif (previous_first == 0) and (previous_second == 1) and (previous_third == 1):
            
            if previous_runs == 1:

                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 1]
                    
            elif previous_runs == 2:

                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 0]

            elif previous_runs == 3:
                
                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]

            else:
                
                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 1]
                
        # Fourth Combination, done
        elif (previous_first == 0) and (previous_second == 0) and (previous_third == 1):
            
            if previous_runs == 1:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 0]

            elif previous_runs == 2:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]
                
            else: 
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 1]
                
        #Fifth Combination, done
        elif (previous_first == 1) and (previous_second == 0) and (previous_third == 0):
            
            if previous_runs == 1:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 1]

            if previous_runs == 2:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]

            else: 
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 0]
                
        # Sixth Combination, done
        elif (previous_first == 1) and (previous_second == 1) and (previous_third == 0):
            
            if previous_runs == 1:
            
                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 1]
                

            elif previous_runs == 2:
                
                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 1]

            elif previous_runs == 3:
                
                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]

            else:
                
                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 1]

        # Seventh Combination, done
        elif (previous_first == 1) and (previous_second == 0) and (previous_third == 1):

            if previous_runs == 1:

                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 0]
                
            elif previous_runs == 2:

                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 1]

            elif previous_runs == 3:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]    

            else:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 1]

        # Eighth Combination, done
        elif (previous_first == 1) and (previous_second == 1) and (previous_third == 1):

            if previous_runs == 1:

                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 1]
                
            elif previous_runs == 2:

                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 1]

            elif previous_runs == 3:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 1]            

    # FieldersChoice can have outs_before = 0 or 1
    # FieldersChoice can have runs_before = 0 or 1
    elif data['PlayResult'].iloc[i-1] == 'FieldersChoice':

        # First Combination, done
        if (previous_first == 0) and (previous_second == 0) and (previous_third == 0):

            if previous_runs == 1 or previous_outs == 1:
            
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]

            else: 
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]

        # Second Combination, done
        elif (previous_first == 0) and (previous_second == 1) and (previous_third == 0):
            
            if previous_runs == 1:

                if previous_outs == 1:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]

                else:
                    
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]
                

            else:

                if previous_outs == 1:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 1]

                else:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 1]
                    
        # Third Combination, done
        elif (previous_first == 0) and (previous_second == 1) and (previous_third == 1):
            
            if previous_runs == 1:

                if previous_outs == 1:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 1]

                else: 

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 1]

            else:

                if previous_outs == 1:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 1]

                else: 

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 1]
                
                
        # Fourth Combination, done
        elif (previous_first == 0) and (previous_second == 0) and (previous_third == 1):
            
            if previous_runs == 1:

                if previous_outs == 1:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]

                else: 

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]

            else:

                if previous_outs == 1:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 1]

                else: 

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 1]
                
        #Fifth Combination, done
        elif (previous_first == 1) and (previous_second == 0) and (previous_third == 0):
            
            if previous_runs == 1:

                if previous_outs == 1:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]

                else: 

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]

            else:

                if previous_outs == 1:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]

                else: 

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 0]
                
        # Sixth Combination, done
        elif (previous_first == 1) and (previous_second == 1) and (previous_third == 0):
            
            if previous_runs == 1:

                if previous_outs == 1:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 0]

                else: 

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 1]
                
            else:

                if previous_outs == 1:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 0]

                elif previous_outs == 2:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]

                else: 

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 1]

        # Seventh Combination, done
        elif (previous_first == 1) and (previous_second == 0) and (previous_third == 1):

            if previous_runs == 1:

                if previous_outs == 1:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]

                else: 

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 0]
                
            else:

                if previous_outs == 1:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 1]

                else: 

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 1]

        # Eighth Combination, done
        elif (previous_first == 1) and (previous_second == 1) and (previous_third == 1):

            if previous_runs == 1:

                if previous_outs == 1:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 0]

                else: 

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 1]

            else:

                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 1]

                
    # Out means outs_before must be >= 1
    # Out means runs_before can equal 0 or 1
    elif data['PlayResult'].iloc[i-1] == 'Out':

        # First Combination, done
        if (previous_first == 0) and (previous_second == 0) and (previous_third == 0):

            if previous_runs == 1 or previous_outs == 1:
            
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]

            else: 
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]

        # Second Combination, done
        elif (previous_first == 0) and (previous_second == 1) and (previous_third == 0):
            
            if previous_runs == 1:

                if previous_outs == 1:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]

                else:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]           


            elif previous_runs == 2:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]   
                
            else:

                if previous_outs == 1:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 0]  

                else:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 0]           
                
        # Third Combination, done
        elif (previous_first == 0) and (previous_second == 1) and (previous_third == 1):
            
            if previous_runs == 1:

                if previous_outs == 1:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 1]

                else: 

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]
                    
            elif previous_runs == 2:

                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]

            else:

                if previous_outs == 1:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 1]

                else: 

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 1]
                
                
        # Fourth Combination, done
        elif (previous_first == 0) and (previous_second == 0) and (previous_third == 1):
            
            if previous_runs == 1:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]

            else:

                if previous_outs == 1:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]

                else: 

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]
                
        #Fifth Combination, done
        elif (previous_first == 1) and (previous_second == 0) and (previous_third == 0):
            
            if previous_runs == 1:
                
                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]

            else:

                if previous_outs == 1:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]

                else:
                    
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]
                
        # Sixth Combination, done
        elif (previous_first == 1) and (previous_second == 1) and (previous_third == 0):
            
            if previous_runs == 1:

                if previous_outs == 1:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 0]

                else:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]
                    
            elif previous_runs == 2:
                
                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]

            else:

                if previous_outs == 1:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 0]

                else:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]
            
                

        # Seventh Combination, done
        elif (previous_first == 1) and (previous_second == 0) and (previous_third == 1):

            if previous_runs == 1:

                if previous_outs == 1:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]

                else: 

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]
                    
            elif previous_runs == 2:
            

                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]
                
            else:

                if previous_outs == 1:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 0]

                elif previous_outs == 2:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]

                else: 

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]

        # Eighth Combination, done
        elif (previous_first == 1) and (previous_second == 1) and (previous_third == 1):

            if previous_runs == 1:

                if previous_outs == 1:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 0]

                else:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]
                    
            elif previous_runs == 2:
                
                if previous_outs == 1:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 0]

                else: 

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]

            elif previous_runs == 3:
                
                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]

            else:
                
                if previous_outs == 1:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 1]

                elif previous_outs == 2:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 0]

                else:
                
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]
                

    # outs_before can equal 1 or 2
    # runs_before can equal 0 or 1
    elif data['PlayResult'].iloc[i-1] == 'Sacrifice': 

        # First Combination, done
        if (previous_first == 0) and (previous_second == 0) and (previous_third == 0):

            data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]


        # Second Combination, done
        elif (previous_first == 0) and (previous_second == 1) and (previous_third == 0):
            
            if previous_runs == 1:
                
                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]         
                
            else:

                if previous_outs == 1:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 1]  

                else:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]           
                
        # Third Combination, done
        elif (previous_first == 0) and (previous_second == 1) and (previous_third == 1):
            
            if previous_runs == 1:

                if previous_outs == 1:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 1]

                else: 

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]

            else:
                
                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 1]             
                
                
        # Fourth Combination, done
        elif (previous_first == 0) and (previous_second == 0) and (previous_third == 1):
            
            data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]
                
        #Fifth Combination, done
        elif (previous_first == 1) and (previous_second == 0) and (previous_third == 0):
            
            if previous_runs == 1:
                
                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]

            else:
                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 0]

                
        # Sixth Combination, done
        elif (previous_first == 1) and (previous_second == 1) and (previous_third == 0):
            
            if previous_runs == 1:

                if previous_outs == 1:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 1]

                else:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]

            else:
                
                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 1]
                            

        # Seventh Combination, done
        elif (previous_first == 1) and (previous_second == 0) and (previous_third == 1):

            if previous_runs == 1:

                if previous_outs == 1:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 0]

                else: 

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]

            else:
                
                if previous_outs == 1:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 1]

                else:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 1]
                    

                
        # Eighth Combination, done
        elif (previous_first == 1) and (previous_second == 1) and (previous_third == 1):

            if previous_runs == 1:

                if previous_outs == 1:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 1]

                elif previous_outs == 2:

                    data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 1]

            else:

                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 1]


    
    # Use PitchCall along with following scenarios scenarios
    # RunsScored = 0 AND OutsOnPlay = 0: BallIntentional, Undefined, BallinDirt
    # RunsScored = 0 or 1 AND OutsOnPlay = 0: Ball, HitByPitch, BallCalled
    # RunsScored = 0 AND OutsOnPlay = 0 or 1: StrikeCalled, FoulBall, StrikeSwinging
    # RunsScored = 0 or 1 AND OutsOnPlay = 0 or 1: 



    # Assuming All Walks can only be one run
    # If not a walk then 0 runs scored
    # Does not account for stolen bases
    elif data['PlayResult'].iloc[i-1] == 'Undefined':

        # First Combination, done
        if (previous_first == 0) and (previous_second == 0) and (previous_third == 0):

            if previous_KorBB == 'Walk':
            
                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0]

            else: 
            
                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]  

        # Second Combination, done
        elif (previous_first == 0) and (previous_second == 1) and (previous_third == 0):
            
            if previous_KorBB == 'Walk':

                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 0]            
                
            else:

                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 0]         
                
        # Third Combination, done
        elif (previous_first == 0) and (previous_second == 1) and (previous_third == 1):
            
            if previous_KorBB == 'Walk':

                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 1]            
                
            else:

                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 1, 1]    
                
                
        # Fourth Combination, done
        elif (previous_first == 0) and (previous_second == 0) and (previous_third == 1):
            
            if previous_KorBB == 'Walk':

                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 1]            
                
            else:

                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [0, 0, 1] 

                
        #Fifth Combination, done
        elif (previous_first == 1) and (previous_second == 0) and (previous_third == 0):
            
            if previous_KorBB == 'Walk':

                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 0]            
                
            else:

                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 0] 
                
        # Sixth Combination, done
        elif (previous_first == 1) and (previous_second == 1) and (previous_third == 0):
            
            if previous_KorBB == 'Walk':

                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 1]            
                
            else:

                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 0] 
            

        # Seventh Combination, done
        elif (previous_first == 1) and (previous_second == 0) and (previous_third == 1):

            if previous_KorBB == 'Walk':

                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 1]            
                
            else:

                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 0, 1] 

        # Eighth Combination, done
        elif (previous_first == 1) and (previous_second == 1) and (previous_third == 1):

            if previous_KorBB == 'Walk':

                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 1]            
                
            else:

                data.loc[i, ['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd']] = [1, 1, 1]

In [51]:
test_1 = data[['PlayResult', 'KorBB', 'PitchCall', 'runners_on_1st', 'runners_on_2nd', 'runners_on_3rd', 'Outs', 'OutsOnPlay', 'RunsScored', 'Balls', 'Strikes', 'PlateLocHeight', 'PlateLocSide']]
pd.set_option('display.max_rows', None)

# Step 1: Calculate the average runs scored for each unique combination of runners_on, batter_count, and outs
run_expectancy = (
    test_1.groupby(['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd', 'Balls', 'Strikes', 'Outs'])
    .agg(run_expectancy=('RunsScored', 'mean'))
    .reset_index()
)

# Step 2: Merge this run expectancy back onto the original dataframe based on the same grouping columns
baseball = test_1.merge(
    run_expectancy,
    on=['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd', 'Balls', 'Strikes', 'Outs'],
    how='left'
)

In [53]:
import itertools

# Step 1: Define all possible combinations of runners, balls, strikes, and outs
combinations = pd.DataFrame(
    list(itertools.product([0, 1], [0, 1], [0, 1], range(4), range(3), range(3))),
    columns=['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd', 'Balls', 'Strikes', 'Outs']
)

# Step 2: Calculate the average runs scored for existing combinations in your data
run_expectancy = (
    baseball.groupby(['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd', 'Balls', 'Strikes', 'Outs'])
    .agg(run_expectancy=('RunsScored', 'mean'))
    .reset_index()
)

# Step 3: Merge all possible combinations with the calculated run expectancy, filling missing values
run_expectancy_matrix = combinations.merge(run_expectancy, 
                                           on=['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd', 'Balls', 'Strikes', 'Outs'],
                                           how='left')

# Step 4: Merge this complete run expectancy matrix back onto the original dataframe
baseball = baseball.merge(run_expectancy_matrix, 
                          on=['runners_on_1st', 'runners_on_2nd', 'runners_on_3rd', 'Balls', 'Strikes', 'Outs'],
                          how='left')

In [55]:
# Step 2: Shift 'run_expectancy' to get the next pitch's run expectancy
baseball['next_run_expectancy'] = baseball['run_expectancy_x'].shift(-1)

# Step 3: Calculate the target variable as (next_run_expectancy + runsscored) - run_expectancy
baseball['target_variable'] = (baseball['next_run_expectancy'] + baseball['RunsScored']) - baseball['run_expectancy_x']

# Step 4: Drop the helper column if not needed
baseball = baseball.drop(columns=['next_run_expectancy'])


In [59]:
# Data is now ready for modeling
baseball.to_csv('data_cleaned.csv', index=False)

15018